In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import selenium
import pandas as pd
import numpy as np
from selenium import webdriver
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

### Q1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [179]:
webD = webdriver.Chrome(r"C:\Users\User\Downloads\chromedriver_win32\chromedriver.exe")
url = "https://www.amazon.in/"
webD.get(url)
search_box = webD.find_element_by_id('twotabsearchtextbox')
search_box
val = input("Enter the product category: ")
search_box.send_keys(val)
search_btn = webD.find_element_by_id('nav-search-submit-button')
search_btn.click()

Enter the product category: guitar


### Q2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [180]:
#Running loop to store the product links in a list
listOflinks =[]
count = 1
while count <4:
    time.sleep(3)
    productInfoList=webD.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']")
    for el in productInfoList:
        listOflinks.append(el.get_attribute('href'))
    link = webD.find_element_by_xpath("//li[@class = 'a-last']//a")
    link.click()
    count += 1

len(listOflinks)

156

In [201]:
BrandName  = []
Nameofproduct = []
Ratings=[]
NoofRatings = []
Price=[]
ReturnExchange = []
ExpectedDelivery  =  []
Availability = []
OtherDetails = []

In [ ]:
for el in listOflinks:
    webD.get(el)
    time.sleep(3)
    title = webD.find_element_by_xpath('//*[@id="productTitle"]').text
    Nameofproduct.append(title)
    BrandName.append(title.split(sep=' ')[0])
    Price.append(webD.find_element_by_id('priceblock_dealprice').text)
    ExpectedDelivery.append(webD.find_element_by_id('ddmDeliveryMessage').text.split(":")[1].replace(" Details",""))
    Availability.append(webD.find_element_by_id('availability').text)
    OtherDetails.append(webD.find_element_by_id('priceblock_dealprice').text)
    details = webD.find_elements_by_xpath('//*[@class="a-size-small a-link-normal a-text-normal"]')
    for i in details:
        if 'Days' in i.text:
            ReturnExchange.append(i.text)
        if 'Warranty' in i.text:
            OtherDetails.append(i.text)
    stars = webD.find_elements_by_xpath('//*[@class="a-size-base"]')
    for i in stars:
        if 'out of 5 stars' in i.text:
            NoofRatings.append(i.text.split("\n")[0])
            Ratings.append(i.text.split("\n")[1])

In [ ]:
df=pd.DataFrame({'BrandName': BrandName, 'Nameofproduct':Nameofproduct, 'Ratings':Ratings, 'NoofRatings': NoofRatings, 'Price': Price, 'ReturnExchange': ReturnExchange, 'ExpectedDelivery': ExpectedDelivery, 
'Availability': Availability, 'OtherDetails':  OtherDetails})

In [ ]:
# saving the dataframe
df.to_csv('file1.csv')

### Q3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [242]:
import os
import selenium
from selenium import webdriver
import time
import io
import requests

In [243]:
def downloadimage(search_element):
    driver = webdriver.Chrome(r"C:\Users\User\Downloads\chromedriver_win32\chromedriver.exe")
    url = "https://images.google.com/"
    driver.get(url)
    search = driver.find_element_by_xpath("//input[@class = 'gLFyf gsfi']")
    search.send_keys(search_element)
    search_btn = driver.find_element_by_xpath("//button[@class = 'Tg7LZd']")
    search_btn.click()
    #Scroll to the end of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)#sleep_between_interactions
    #Locate the images to be scraped from the current page 
    imgResults = driver.find_elements_by_xpath("//img[contains(@class,'Q4LuWd')]")

    #Click on each Image to extract its corresponding link
    img_urls = []
    
    for i in  range(0,len(imgResults)):
        img=imgResults[i]
        try:
            img.click()
            time.sleep(2)
            actual_images = driver.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'https' in actual_image.get_attribute('src'):
                    img_urls.append(actual_image.get_attribute('src'))
        except ElementClickInterceptedException or ElementNotInteractableException as err:
            print(err)
    
    #Saving the images to current directory
    count = 0
    for i in range(len(img_urls)):
        reponse = requests.get(img_urls[i])
        if reponse.status_code == 200:
            with open(f"{search_element}{count+1}.jpg","wb") as file:
                file.write(reponse.content)
                count += 1
    return

In [244]:
downloadimage("fruits")
downloadimage("cars")
downloadimage("Machine Learning")

### 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [235]:
driver = webdriver.Chrome(r"C:\Users\User\Downloads\chromedriver_win32\chromedriver.exe")
time.sleep(3)

url = "https://www.flipkart.com/"
driver.get(url)

button = driver.find_element_by_xpath("//button[@class = '_2KpZ6l _2doB4z']")
button.click()

#locating the search bar
search_bar=driver.find_element_by_class_name("_3704LK")
search_bar.send_keys('realme')

time.sleep(2)
#locating the button and clicking it toh search for sunglasses
button=driver.find_element_by_class_name('L0Z3Pu')
button.click()

urls  = []
for j in driver.find_elements_by_xpath('//a[@class="_1fQZEK"]'):
    urls.append(j.get_attribute('href'))
    
#creating the empty lists
BrandName = []
Smartphonename = []
Colour = []
RAM = []
Storage = []
PrimaryCamera = []
SecondaryCamera = []
DisplaySize = []
DisplayResolution = []
Processor = []
ProcessorCores = []
BatteryCapacity = []
Price = []

In [239]:
for i in urls:
    driver.get(i)
    BrandName.append(driver.find_element_by_class_name('B_NuCI').text.split(" ")[0])
    Smartphonename.append(driver.find_element_by_class_name('B_NuCI').text)
    details1 = driver.find_elements_by_xpath("//div[@class='_3k-BhJ']")
    details1_l = []
    for i in details1:
        details1_l.append(i.text.split("\n"))
    for i in range(len(details1_l)):
        for j in range(len(details1_l[i])):
            if 'Color' in details1_l[i][j]:
                Colour.append(details1_l[i][j+1])
        for j in range(len(details1_l[i])):
            if 'RAM' in details1_l[i][j]:
                RAM.append(details1_l[i][j+1])
        for j in range(len(details1_l[i])):
            if 'Internal Storage' in details1_l[i][j]:
                Storage.append(details1_l[i][j+1])
        for j in range(len(details1_l[i])):
            if 'Primary Camera' in details1_l[i][j]:
                PrimaryCamera.append(details1_l[i][j+1])
        for j in range(len(details1_l[i])):
            if 'Secondary Camera' in details1_l[i][j]:
                SecondaryCamera.append(details1_l[i][j+1])
        for j in range(len(details1_l[i])):
            if 'Display Size' in details1_l[i][j]:
                DisplaySize.append(details1_l[i][j+1])
        for j in range(len(details1_l[i])):
            if 'Resolution' in details1_l[i][j]:
                DisplayResolution.append(details1_l[i][j+1])
        for j in range(len(details1_l[i])):
            if 'Processor Type' in details1_l[i][j]:
                Processor.append(details1_l[i][j+1])
        for j in range(len(details1_l[i])):
            if 'Processor Core' in details1_l[i][j]:
                ProcessorCores.append(details1_l[i][j+1])
        for j in range(len(details1_l[i])):
            if 'Battery Capacity' in details1_l[i][j]:
                BatteryCapacity.append(details1_l[i][j+1])

In [ ]:
df=pd.DataFrame({'BrandName':BrandName,'Smartphonename': Smartphonename,
                 'Colour': Colour, 'RAM': RAM, 'Storage':Storage,
                 'PrimaryCamera': PrimaryCamera, 'SecondaryCamera': SecondaryCamera,
                 'DisplaySize' :DisplaySize, 'DisplayResolution': DisplayResolution,
                 'Processor': Processor, 'ProcessorCores' :ProcessorCores,
                 'BatteryCapacity': BatteryCapacity, 'Price': Price})

In [ ]:
# saving the dataframe
df.to_csv('flipkart.csv')

### 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [172]:
webD = webdriver.Chrome(r"C:\Users\User\Downloads\chromedriver_win32\chromedriver.exe")
url = "https://www.google.com/maps/@19.3164803,84.8046344,15z"
webD.get(url)
search_box = webD.find_element_by_id('searchboxinput')
search_box
val = input("Enter the city: ")
search_box.send_keys(val)
search_btn = webD.find_element_by_id('searchbox-searchbutton')
search_btn.click()

Enter the city: delhi


In [173]:
str = webD.current_url.split("/")

In [174]:
for i in str:
    if '@' in i:
        latlong = i.replace("@","")

In [176]:
latitude = latlong.split(",")[0]
longitude  = latlong.split(",")[1]

In [177]:
latitude

'28.6466772'

In [178]:
longitude

'76.8130591'

### 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [269]:
webD = webdriver.Chrome(r"C:\Users\User\Downloads\chromedriver_win32\chromedriver.exe")
url = "https://trak.in/india-startup-funding-investment-2015/"
webD.get(url)

In [270]:
sep2020 = webD.find_element_by_id("tablepress-50_wrapper")

In [274]:
aug2020 = webD.find_element_by_id("tablepress-49_wrapper")

In [ ]:
jul2020 = webD.find_element_by_id("tablepress-48_wrapper")

### Q7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [190]:
driver = webdriver.Chrome(r"C:\Users\User\Downloads\chromedriver_win32\chromedriver.exe")
url = 'https://www.digit.in/top-products/best-gaming-desktop-in-india-3605.html'
driver.get(url)

In [217]:
title = []
title_e = driver.find_elements_by_xpath("//div[@class='TopNumbeHeading active sticky-footer']")
for i in title_e:
    if i.text is None :
        title.append("--") 
    else:
        title.append(i.text)

In [218]:
title

['1.\nMSI INFINITEX PLUS (9SF-270US)',
 '2.\nROG STRIX GA35 (G35DX-IN003T)',
 '3.\nPREDATOR ORION 9000 PO9-900',
 '4.\nLEGION T530 (90L300NYIN)',
 '5.\nINSPIRON G5 GAMING DESKTOP',
 '6.\nOMEN X COMPACT DESKTOP (P1000-013IN)']

In [219]:
desc = []
desc_e = driver.find_elements_by_xpath("//div[@class='Section-center']//p")
for i in desc_e:
    if i.text is None:
        desc.append("--")
    elif(len(i.text)>12):
        desc.append(i.text)

In [220]:
desc

["MSI opts for an Intel Core i9-9900K for the processor while using the NVIDIA GeForce RTX 2080 Ti for their gaming desktop. The MSI Infinite X Plus 9SF-270US comes in a mid-tower form factor and has 32 GB of RAM to take care of all your multi-tasking needs and for storage they're giving a 512GB NVMe SSD as primary storage and they're throwing in an additional 2TB hard drive for additional storage. Since the Intel Core i9-9900K performs better than the AMD Ryzen 9 3950X in gaming, this combination seems to be apt. For all this, they have quite a steep price tag.",
 "The ASUS ROG Strix GA35 is one of the most feature-laden desktops on offer in India. It uses a Ryzen 9 3950X which is a 16-Core processor that is neck-and-neck with the Intel Core i9-9900K for gaming performance. The GA35 uses an X570 platform which makes it PCIe 4.0 Ready and Gen4 NVMe SSDs can really take advantage of that. For graphics, the GA35 has an NVIDIA GeForce RTX 2080 Ti which is still the best gaming graphics ca

In [221]:
desc_new = []
for i in range(len(desc)):
    if ('OUT OF STOCK' in desc[i]) or ('AVAILABLE' in desc[i]):
        continue
    else:
        desc_new = desc[i]

In [222]:
desc_new

"HP has plenty of gaming laptops under the OMEN brand but no desktops. The only one we could find happens to be the compact desktop which doubles up as a VR backpack. This unit is powered by an Intel Core i7-7820HK and has an NVIDIA GTX 1080 for the graphics card. On the memory front, you're getting 16 GB of DDR4 RAM and for storage, this machine has a 1 TB SSD. HP's newer OMEN 15 and OMEN X 25 laptops are more powerful than this desktop."

In [223]:
availability = []
record = []
l = len(desc)
for i in range(l):
    if(len(desc[i])<5):
        print(len(desc[i]))
        availability.append(i)
        record.append(i)

In [224]:
len(desc[2])

676

In [225]:
merchant = []
merchant_e = driver.find_elements_by_xpath("//div[@class='merchantdatalist1']")
for i in merchant_e:
    if (i.text is None) or (len(i.text)==0):
        merchant.append("--")
    else:
        merchant.append(i.text.replace("\nBUY NOW",""))

In [226]:
merchant

['OUT OF STOCK ₹ 539990',
 '--',
 'OUT OF STOCK ₹ 452000',
 'AVAILABLE ₹ 132746',
 'OUT OF STOCK ₹ 169990',
 'AVAILABLE ₹ 298827']

In [227]:
df = pd.DataFrame(merchant)

In [228]:
df

,0
0,OUT OF STOCK ₹ 539990
1,--
2,OUT OF STOCK ₹ 452000
3,AVAILABLE ₹ 132746
4,OUT OF STOCK ₹ 169990
5,AVAILABLE ₹ 298827


In [229]:
df[['Status of availability', 'Price']] = df[0].str.split("₹",expand=True)

In [230]:
df.drop(columns=0, inplace= True)

In [231]:
df["Description"] = desc

In [232]:
len(title)

6

In [233]:
df["Laptop Model"] = title

In [235]:
df

,Status of availability,Price,Description,Laptop Model
0,OUT OF STOCK,539990,MSI opts for an Intel Core i9-9900K for the pr...,1.\nMSI INFINITEX PLUS (9SF-270US)
1,--,None,The ASUS ROG Strix GA35 is one of the most fea...,2.\nROG STRIX GA35 (G35DX-IN003T)
2,OUT OF STOCK,452000,"Acer's top of the line gaming desktop, the Ori...",3.\nPREDATOR ORION 9000 PO9-900
3,AVAILABLE,132746,Lenovo takes a more price conscious approach w...,4.\nLEGION T530 (90L300NYIN)
4,OUT OF STOCK,169990,The new Dell G5 gaming desktop uses an Intel C...,5.\nINSPIRON G5 GAMING DESKTOP
5,AVAILABLE,298827,HP has plenty of gaming laptops under the OMEN...,6.\nOMEN X COMPACT DESKTOP (P1000-013IN)
